<a href="https://colab.research.google.com/github/yifengd/adversarial-nlp/blob/main/defenses/wdr/Training_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers >> /dev/null

# WDR Training and Testing (with Improvements)

This script can be used to train and test the classifier on original and adversarial samples.

In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
import time
import importlib
from copy import copy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
MODE = "TEST" # TRAIN

MULTIWORD_MASK_N = 2
MLM_MASK = False

# Loading and transforming data into logits differences

The first step is transforming our dataframe into logits differences for each original and adversarial sentence. For this, it is required to execute the model for each sentence with substitutions as explained in the paper.

In [4]:
SAMPLES_PATH = '/content/drive/MyDrive/AdversarialXAI/Adversarial Samples'

# Print available setups for testing
for i in os.listdir(SAMPLES_PATH):
    if not i.startswith('.'): # Don't print system files
        print(i)

Older attacks
Raw from benchmark
Old
ag-news_2000_train_styleadv_distilbert.csv
ag-news_dev_2000_styleadv_distilbert.csv
imdb_test_100_styleadv_distilbert.csv
imdb_train_1000_styleadv_distilbert.csv
sst2_test_pruthi_bert.csv
sst2_train_2000_styleadv_bert.csv
sst2_train_pruthi_bert.csv
sst2_val_pruthi_bert.csv
sst2_val_styleadv_bert.csv


In [5]:
# Select the configuration for training
dataset_path = 'sst2_train_2000_styleadv_bert.csv' # or 'agnews_pwws_distilbert.csv'

In [6]:
# Obtain model from test config
model_arch = dataset_path.replace(".csv", "").split('_')[-1]
dataset = dataset_path.split('_')[0]
if dataset == "sst2":
  dataset = "SST-2"
print("Model architecture:", model_arch)
print("Dataset:", dataset)

Model architecture: bert
Dataset: SST-2


In [7]:
def load_textattack_local_model(model_arch, dataset):
    
    def load_module_from_file(file_path):
        """Uses ``importlib`` to dynamically open a file and load an object from
        it."""
        temp_module_name = f"temp_{time.time()}"

        spec = importlib.util.spec_from_file_location(temp_module_name, file_path)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        return module
    
    m = load_module_from_file(f'../{model_arch}_{dataset}_textattack.py')
    model = getattr(m, 'model')
    
    return model, None

In [8]:
def load_hugging_face_model(model_arch, dataset):
    # Import the model used for generating the adversarial samples.
    # Correctly, set up imports, model and tokenizer depending on the model you generated the samples on.
    
    if model_arch == 'distilbert':
        from transformers import DistilBertConfig as config, DistilBertTokenizer as tokenizer, AutoModelForSequenceClassification as auto_model
    elif model_arch == 'bert':
        from transformers import BertConfig as config, BertTokenizer as tokenizer, AutoModelForSequenceClassification as auto_model
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    tokenizer = tokenizer.from_pretrained(f"textattack/{model_arch}-base-uncased-{dataset}")
    model = auto_model.from_pretrained(f"textattack/{model_arch}-base-uncased-{dataset}").to(device)
    
    return model, tokenizer

In [9]:
# Models available in hugging-face are executed differently from LSTM and CNN. Choose automatically the configuration and load model + tokenizer.
textattack_local_models = ['lstm', 'cnn']

if model_arch in textattack_local_models:
    hugging_face_model = False
    model, tokenizer = load_textattack_local_model(model_arch, dataset)

else:
    hugging_face_model = True
    model, tokenizer = load_hugging_face_model(model_arch, dataset)

# Loading data

Read into a dataframe your original and adversarial samples.

In [10]:
# Read the desired csv file previously generated
df = pd.read_csv(f'{SAMPLES_PATH}/{dataset_path}')
df.shape

(1972, 5)

In [11]:
# Select first entries. Only 3000 will be used but we leave room for false adversarial sentences that will be filtered out later and test set. We reduce size because computations are expensive.
# In real setup, the whole file was considered and fixed train and test sets were produced.
df = df.head(7000)

In [12]:
# Create batches of non-adversarial sentences
# For big models such as BERT, we must divide our input in smaller batches.
n = 256 # Size of each batch.
batches = [list(df.original_text.values)[i:i + n] for i in range(0, len(df.original_text.values), n)]

In [13]:
batches[0][0]

' elegant visual sense  '

In [14]:
# Generate predictions for all non-adversarial sentences in our dataset
outputs = []

if hugging_face_model is True: # Use tokenizer and hugging face pipeline
    for b in batches: 
        input = tokenizer(b, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            output = model(**input)
            outputs.append(output.logits.cpu().numpy())
            del input
            torch.cuda.empty_cache()

else: # Use local model by simply predicting without tokenization
    for b in batches: 
        output = model(b)
        outputs.append(output)

In [15]:
# Obtain non-adversarial predictions
outputs_flatten = [item for sublist in outputs for item in sublist]
predictions = [np.argmax(i) for i in outputs_flatten]

# Include prediction for these classes in our DataFrame
df['original_class_predicted'] = predictions

In [16]:
# Repeat process for adversarial sentences
n = 256
batches = [list(df.adversarial_text.values)[i:i + n] for i in range(0, len(df.adversarial_text.values), n)]

In [17]:
# Generate predictions for all non-adversarial sentences in our dataset
outputs = []

if hugging_face_model is True: # Use tokenizer and hugging face pipeline
    for b in batches: 
        input = tokenizer(b, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            output = model(**input)
            outputs.append(output.logits.cpu().numpy())
            del input
            torch.cuda.empty_cache()

else: # Use local model by simply predicting without tokenization
    for b in batches: 
        output = model(b)
        outputs.append(output)

In [18]:
# Obtain adversarial predictions
outputs_flatten = [item for sublist in outputs for item in sublist]
predictions = [np.argmax(i) for i in outputs_flatten]

# Include prediction for these classes in our DataFrame
df['adversarial_class_predicted'] = predictions

In [19]:
# Select only those sentences for which there was actually a change in the prediction
correct = df[(df['original_class_predicted'] != df['adversarial_class_predicted'])]

In [20]:
# Update dataframe and keep only adversarial samples
df = correct
len(df)

926

# Obtain logits
Once we have the predictions and actually adversarial sentences, we generate the logits differences

In [21]:
original_samples = df.original_text.values
adversarial_samples = df.adversarial_text.values

In [22]:
# Concatenate all original samples and their predictions
x = np.concatenate((original_samples, adversarial_samples))
y = np.concatenate((np.zeros(len(original_samples)), np.ones(len(adversarial_samples))))

In [23]:
def obtain_logits(samples, batch_size, model, tokenizer):
    """
    For given samples and model, compute prediction logits.
    Input data is splitted in batches.
    """
    batches = [samples[i:i + batch_size] for i in range(0, len(samples), batch_size)]
    logits = []

    for i, b in enumerate(batches):
        print("{}/{}".format(i+1, len(batches)))
        if hugging_face_model:
            with torch.no_grad():
                input = tokenizer(list(b), return_tensors="pt", padding=True, truncation=True).to(device)
                logits.append(model(**input).logits.cpu().numpy())
        else:
            logits.append(model(b))

    return logits

In [24]:
# Compute logits for original sentences
batch_size = 350
original_logits = obtain_logits(original_samples, batch_size, model, tokenizer)
original_logits = np.concatenate(original_logits).reshape(-1, original_logits[0].shape[1])

1/3
2/3
3/3


In [25]:
torch.cuda.empty_cache()

In [26]:
# Compute logits for adversarial sentences
batch_size = 350
adversarial_logits = obtain_logits(adversarial_samples, batch_size, model, tokenizer)
adversarial_logits = np.concatenate(adversarial_logits).reshape(-1, adversarial_logits[0].shape[1])

1/3
2/3
3/3


In [27]:
torch.cuda.empty_cache()

In [28]:
# Concatenate all logits
logits = np.concatenate((original_logits, adversarial_logits))

In [29]:
# Shuffle data
import random
c = list(zip(x, y, logits))
random.shuffle(c)
x, y, logits = zip(*c)
[s for s in x if len(s.split()) == 1]

[' ridiculousness  ',
 ' profane  ',
 ' sappiness  ',
 ' aplenty  ',
 ' spiffy  ',
 ' virtuous  ',
 ' lameness  ',
 ' old  ',
 ' miike  ',
 ' irritates  ',
 ' disoriented  ',
 ' uncompromising  ',
 ' shines  ',
 ' warmth  ',
 ' ethereal  ',
 ' embarrassed  ',
 ' sleep-inducingly  ',
 ' paint-by-numbers  ',
 ' errors  ',
 ' laughs  ',
 ' exploitation  ',
 ' swedish  ',
 ' faithful  ',
 ' allusions  ',
 ' discordant  ',
 ' faithful................................................................................................................................................................................................................................................................................................................................ ',
 ' falls  ',
 ' affectingly  ',
 ' fiascos  ',
 ' delicate  ',
 ' desperate  ',
 ' deception  ',
 ' flatula  ',
 ' ace  ',
 ' peak  ',
 ' spontaneous  ',
 ' metaphor  ',
 ' cliffhanger  ',
 ' competently  ',
 ' overwhelming  ',
 ' militarism  

## Computing logits difference

This is a key step implemented. The main idea is:
* For each sentence, replace each word by the `[UNK]` token and compute prediction logits
* Using these logits, we can easily compute the saliency of the word as presented in the report.
* Then, we sort words by descending saliency.
* Finally, compute logits difference for each replacement. This difference is computed as `Logit from class predicted for the whole sentence - Highest remaining logit`

More details on these derivations are found in the paper.

In [30]:
from transformers import pipeline
pipe = pipeline('fill-mask', model=f"distilbert-base-uncased")

In [31]:
def compute_logits_difference(x, logits, y, model, tokenizer, idx, max_sentence_size=512, num_masked=1, mask_fill=None):
    n_classes = len(logits[idx])
    predicted_class = np.argmax(logits[idx]) # Predicted class for whole sentence using previously computed logits
    class_logit = logits[idx][predicted_class] # Store this origianl prediction logit

    split_sentence = x[idx].split(' ')[:max_sentence_size] # The tokenizer will only consider 512 words so we avoid computing innecessary logits

    new_sentences = []

    if mask_fill != None and num_masked > 1:
      raise NotImplementedError

    if mask_fill != None:
      for i, word in enumerate(split_sentence):
        new_sentence = copy(split_sentence)
        new_sentence[i] = '[MASK]'
        new_sentence = ' '.join(new_sentence)
        new_sentences.append(new_sentence)
    else:
      # Here, we replace each word by [UNK] and generate all sentences to consider
      for i, word in enumerate(split_sentence[:len(split_sentence) - num_masked + 1]):
          new_sentence = copy(split_sentence)
          for k in range(num_masked): 
            new_sentence[i + k] = '[UNK]'
          new_sentence = ' '.join(new_sentence)
          new_sentences.append(new_sentence)
    
    if len(new_sentences) == 0:
      return torch.zeros(0, 1).to(device), torch.Tensor([y[idx]]).to(device)
    
    try: 
      if mask_fill != None:
        new_sentences = [m[0]['sequence'] for m in mask_fill(new_sentences)]
    except KeyError:
      print(new_sentences)

    # print(new_sentences[-1])

    # We cannot run more than 350 predictions simultaneously because of resources.
    # Split in batches if necessary.
    # Compute logits for all replacements.
    if len(new_sentences) > 200:
        logits = []
        batches = [new_sentences[i:i + 200] for i in range(0, len(new_sentences), 200)]
        for b in batches:
            if hugging_face_model: # Use hugging face predictions
                batch = tokenizer(b, return_tensors="pt", padding=True, truncation=True).to(device)
                with torch.no_grad():
                    logits.append(model(**batch).logits)
            else:
                logits.append(model(b).to(device))
      
        if hugging_face_model:
            logits = torch.cat(logits)
        else:
            logits = np.concatenate( logits, axis=0 )
            logits = torch.Tensor(logits)
    
    else: # There's no need to split in batches
        if hugging_face_model:
            batch = tokenizer(new_sentences, return_tensors="pt", padding=True, truncation=True).to(device)
            with torch.no_grad():
                logits = model(**batch).logits
            del batch
        else:
            logits = model(new_sentences)
            logits = torch.Tensor(logits)


    # Compute saliency
    saliency = (class_logit - logits[:,predicted_class]).reshape(-1, 1) # NUM_SENTS x 1

    # Append to logits for sorting
    data = torch.cat((logits, saliency), 1) # NUM_SENTS x NUM_CLASSES + 1

    # Sort by descending saliency
    data = torch.stack(sorted(data, key=lambda a: a[n_classes], reverse=True))

    # Remove saliency
    data = data[:, :n_classes] # NUM_SENTS x NUM_CLASSES

    # Fix order: originallly predicted class, other classes
    order = [predicted_class] + [i for i in range(n_classes) if i!=predicted_class]
    data = torch.index_select(data, 1, torch.LongTensor(order).to(device))  # NUM_SENTS x NUM_CLASSES

    # Compute difference between predicted class (always first column) and higher remaining logit
    data = data[:, :1].flatten() - torch.max(data[:, 1:], dim=1).values.flatten() # NUM_SENTS

    del saliency
    torch.cuda.empty_cache()

    # Return only logits difference
    return data.reshape(-1, 1), torch.Tensor([y[idx]]).to(device) # NUM_SENTS x 1

In [32]:
def compute_logits_difference_padding(x, logits, y, model, tokenizer, idx, target_size=512, mask_fill=None, num_masked=1):
    """
    This function provides a wrapper for compute_logits_difference and includes padding to computations.
    """
    data, y = compute_logits_difference(x, logits, y, model, tokenizer, idx, target_size, mask_fill=mask_fill, num_masked=num_masked)
    data_size = min(512, data.shape[0])
    target = torch.zeros(target_size, 1).to(device)
    target[:data_size, :] = data

    return target, y

In [33]:
from torch.utils.data import Dataset, DataLoader
import sys
from torch.autograd import Variable

class Text(Dataset):
    """
    Dataloader following torch details. Each time we get an item, we will compute
    the logits difference.
    """
    def __init__(self, x , logits, y, model, tokenizer, train=True, max_sentence_size=512, multiword_mask=1, mlm_mask=False):
        self.logits = logits
        self.y = y
        self.x = x
        self.model = model
        self.tokenizer = tokenizer
        self.max_sentence_size = max_sentence_size
        self.multiword_mask = multiword_mask
        self.mlm_mask = mlm_mask

        if multiword_mask > 1 and mlm_mask:
          raise NotImplementedError()

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        data_components = []

        for i in range(1, self.multiword_mask + 1):
          data, y = compute_logits_difference_padding(self.x, self.logits, self.y, self.model, self.tokenizer, idx, self.max_sentence_size, num_masked=i)
          data = data[:, :1].unsqueeze(0)  # 1 x 512 x 1

          data_components.append(data)
        
        if self.mlm_mask:
          data2, _ = compute_logits_difference_padding(self.x, self.logits, self.y, self.model, self.tokenizer, idx, self.max_sentence_size, num_masked=1, mask_fill=pipe)
          data2 = data2[:, :1].unsqueeze(0)

          data_components.append(data2)
        
        if len(data_components) == 1:
          return data_components[0], y, self.x[idx]
        else:
          return torch.cat(data_components, dim=1), y, self.x[idx]

In [34]:
# torch.multiprocessing.set_start_method('spawn', force=True)

# Create the dataloader
train_ds = Text(x, logits, y, model, tokenizer, multiword_mask=MULTIWORD_MASK_N, mlm_mask=MLM_MASK)
train_loader = DataLoader(dataset=train_ds, batch_size=256, shuffle=True)

In [35]:
# Define the target DataFrame to structure our data.
# It has a column for each input dimension (up to 512) and 
# it also includes whether it is adversarial or not (y_label) and the sentence from which the logits where extracted

dimensions = MULTIWORD_MASK_N * 512
if MLM_MASK:
  dimensions += 512

data_all = pd.DataFrame(columns=[i for i in range(dimensions)]+['y_label', 'sentence'])

In [36]:
# Generate logits difference by running the loader.
for i, (data, y_label, sentence) in enumerate(train_loader):
    print("{}/{} - {}\n".format(i, len(train_loader), i/len(train_loader)))
    for v in range(len(data)):
        # Structure data and include in dataframe
        row = np.append(data[v].cpu().numpy().reshape(1,-1), np.array([y_label[v].item(), sentence[v]]))
        data_all = data_all.append(pd.DataFrame([row], columns=list(data_all)), ignore_index=True)

0/8 - 0.0

1/8 - 0.125

2/8 - 0.25

3/8 - 0.375

4/8 - 0.5

5/8 - 0.625

6/8 - 0.75

7/8 - 0.875



In [37]:
data_all

,0,1,2,3,4,5,6,7,8,9,...,1016,1017,1018,1019,1020,1021,1022,1023,y_label,sentence
0,-5.8796806,-1.7704885,-1.2715987,1.305377,2.0623434,5.042874,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"unwelcome, but not unpleasant"
1,0.48988494,1.7928007,2.7327037,2.785447,3.0047982,3.5616093,4.288803,4.3167367,4.362954,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,dealing with it will be the end
2,3.11653,5.0461254,5.0902324,5.204444,5.212596,5.397453,5.4384794,5.483839,5.418788,5.5473948,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,future lizard endeavors will need to adhere m...
3,-5.165555,-3.7377172,-1.6726873,-1.6127242,-1.3955364,-0.82567525,-0.9090843,-0.28754684,-0.004408058,-0.018685993,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,translation...... a routine scary-eyed horror...
4,5.643599,6.098986,6.1851697,6.4314985,6.548147,6.548147,6.657836,6.6529326,6.7026424,6.8727207,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,there 's nothing remotely topical or sexy her...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847,2.9246135,3.183181,4.2413945,4.8510456,4.8510456,5.279681,5.539399,6.3687105,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,recovering from its demented premise
1848,0.3149402,5.694725,5.7139997,5.8302155,5.939002,6.000313,6.498698,6.4615154,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"the criticism is damn, damn it."
1849,1.2095178,3.9293602,4.2312055,5.3438396,5.5033474,6.2454534,6.2620864,7.483782,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,dilates the pleasure of watching them.
1850,1.6187952,5.3177342,5.646173,5.7889147,6.1031427,6.2118397,6.219294,6.225162,6.3867736,6.5153356,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"has a certain ghoulish fascination , and gene..."


In [38]:
if MODE == "TRAIN":
  # Divide train and val set
  # Make an ad-hoc val set here
  data_val = data_all.tail(200)
  data_train = data_all.head(len(data_all) - 200)

  y = data_train['y_label'].values
  x = data_train.drop(columns=['y_label', 'sentence']).values

  y_test = data_val['y_label'].values
  x_test = data_val.drop(columns=['y_label', 'sentence']).values

## Model Testing

Use for testing an existing model

In [39]:
### TESTING MODE

from sklearn.metrics import classification_report, confusion_matrix
import pickle

if MODE == "TEST":
  Y_TEST = data_all['y_label'].values
  X_TEST = data_all.drop(columns=['y_label', 'sentence']).values

  with open("/content/drive/MyDrive/AdversarialXAI/Classifiers/WDR/sst2_pruthi_randomforest_distilbertmask.pickle", 'rb') as f:
    cls = pickle.load(f)

  print(cls)

  preds = cls.predict(X_TEST)

  print(classification_report(Y_TEST, preds, digits=3))

# Model training and comparison

We train different models and compare their performance.

### Random forest

In [40]:
from sklearn.ensemble import RandomForestClassifier

# Create the model using best parameters found
model = RandomForestClassifier(n_estimators=1600,
                               min_samples_split=10,
                               min_samples_leaf=2,
                               max_features='auto',
                               max_depth=None, 
                               bootstrap = True)
# Fit on training data
model.fit(x, y)

RandomForestClassifier(min_samples_leaf=2, min_samples_split=10,
                       n_estimators=1600)

In [41]:
# Actual class predictions
rf_predictions = model.predict(x_test)

In [42]:
np.sum(rf_predictions==y_test)/len(y_test)

0.745

In [43]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, rf_predictions, digits=3))
print(confusion_matrix(y_test, rf_predictions))

              precision    recall  f1-score   support

         0.0      0.769     0.748     0.758       107
         1.0      0.719     0.742     0.730        93

    accuracy                          0.745       200
   macro avg      0.744     0.745     0.744       200
weighted avg      0.746     0.745     0.745       200

[[80 27]
 [24 69]]


### XGBoost

Best performing model. Hyperparamter tuning done with Dataiku.

In [44]:
import xgboost as xgb

In [45]:
xgb_classifier = xgb.XGBClassifier(
                    max_depth=3,
                    learning_rate=0.34281802,
                    gamma=0.6770816,
                    min_child_weight=2.5520658,
                    max_delta_step=0.71469694,
                    subsample=0.61460966,
                    colsample_bytree=0.73929816,
                    colsample_bylevel=0.87191725,
                    reg_alpha=0.9064181,
                    reg_lambda=0.5686102,
                    n_estimators=29,
                    silent=0,
                    nthread=4,
                    scale_pos_weight=1.0,
                    base_score=0.5,
                    missing=None,
                  )

In [46]:
xgb_classifier.fit(x, y)

XGBClassifier(colsample_bylevel=0.87191725, colsample_bytree=0.73929816,
              gamma=0.6770816, learning_rate=0.34281802,
              max_delta_step=0.71469694, min_child_weight=2.5520658,
              n_estimators=29, nthread=4, reg_alpha=0.9064181,
              reg_lambda=0.5686102, scale_pos_weight=1.0, silent=0,
              subsample=0.61460966)

In [47]:
xgb_predictions = xgb_classifier.predict(x_test)

In [48]:
print(classification_report(y_test, xgb_predictions, digits=3))
print(confusion_matrix(y_test, xgb_predictions))

              precision    recall  f1-score   support

         0.0      0.767     0.738     0.752       107
         1.0      0.711     0.742     0.726        93

    accuracy                          0.740       200
   macro avg      0.739     0.740     0.739       200
weighted avg      0.741     0.740     0.740       200

[[79 28]
 [24 69]]


### Saving model

Choose a model to save

In [ ]:
import pickle
# pickle.dump(model, open("/content/drive/MyDrive/AdversarialXAI/Classifiers/WDR/sst2_pruthi_randomforest_distilbertmask.pickle", "wb"))

In [ ]:
# xgb_classifier.save_model("/content/drive/MyDrive/AdversarialXAI/Classifiers/WDR/sst2_styleadv_2000_xgb_distilbertmask.json")

### AdaBoost classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
abc = AdaBoostClassifier()

In [ ]:
abc.fit(x, y)

In [ ]:
abc_predictions = abc.predict(x_test)

In [ ]:
np.sum(abc_predictions==y_test)/len(y_test)

In [ ]:
print(classification_report(y_test, abc_predictions, digits=3))
print(confusion_matrix(y_test, abc_predictions))

### LightGBM

In [ ]:
import lightgbm as lgb

In [ ]:
parameters = {
    'objective': 'binary',
    'application': 'binary',
    'metric': ['binary_logloss'],
    'num_leaves': 35,
    'learning_rate': 0.13,
    'verbose': 1
}

In [ ]:
train_data = lgb.Dataset(x, label=y)
test_data = lgb.Dataset(x_test, label=y_test)

In [ ]:
lgbm_classifier = lgb.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=300)

In [ ]:
y_hat = lgbm_classifier.predict(x_test)

In [ ]:
y_hat.round()

In [ ]:
np.sum(y_hat.round()==y_test)/len(y_test)

In [ ]:
y_test = y_test.astype(np.float)

In [ ]:
print(classification_report(y_test, y_hat.round(), digits=3))
print(confusion_matrix(y_test, y_hat.round()))

### SVM

In [ ]:
from sklearn.svm import SVC
svm_clf = SVC(C=9.0622635,
          kernel='rbf',
          gamma='scale',
          coef0=0.0,
          tol=0.001,
          probability=True,
          max_iter=-1)

In [ ]:
svm_clf.fit(x, y)

In [ ]:
svm_pred = svm_clf.predict(x_test)

In [ ]:
svm_pred = svm_pred.astype(np.float)

In [ ]:
np.sum(svm_pred.round()==y_test)/len(y_test)

In [ ]:
print(classification_report(y_test, svm_pred.round(), digits=3))
print(confusion_matrix(y_test, svm_pred.round()))

### Perceptron NN

In [ ]:
from torch.utils.data import Dataset, DataLoader
import sys
from torch.autograd import Variable

class Text(Dataset):
    def __init__(self, x , y):
        self.y = y
        self.x = x

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        data = torch.tensor(self.x[idx].astype('float32')).to(device)
        y = torch.tensor(self.y[idx].astype('float32')).unsqueeze(0).to(device)
        return data, y

In [ ]:
train_ds = Text(x, y)
train_loader = DataLoader(dataset=train_ds, batch_size=128, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class BasicModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BasicModel, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim  = output_dim

        self.fc1 = torch.nn.Linear(self.input_dim, self.hidden_dim)
        self.fc2 = torch.nn.Linear(self.hidden_dim, 1)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return self.sigmoid(x)

In [ ]:
basic_classifier = BasicModel(input_dim=512*1, hidden_dim=50, output_dim=1).to(device)
c = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(basic_classifier.parameters(), lr=0.001)

train_loss_history = []
val_acc_history = []

In [ ]:
iter_per_epoch = len(train_loader)
num_epochs = 30
initial_epoch = 1
log_nth = 2
storing_frequency = 15
checkpoints_path = "/content/drive/MyDrive/ExplainableAI/Model/Saliency/checkpoints"

for epoch in range(initial_epoch, initial_epoch+num_epochs):
    basic_classifier.train()
    epoch_losses = []
    for i, (data, y_label) in enumerate(train_loader):
      optimizer.zero_grad()
      out = basic_classifier(data)
      loss = c(out, y_label)
      epoch_losses.append(loss.item())
      loss.backward()
      optimizer.step()

      if (i+1) % log_nth == 0:        
          print ('Epoch [{}/{}], Step [{}/{}], Loss for last {} batches: {:.4f}' 
                  .format(epoch, num_epochs, i+1, iter_per_epoch, log_nth, np.mean(np.array(epoch_losses[-log_nth:]))))
          #print_time()
      
      if (i+1) % storing_frequency == 0:        
          print('Storing with loss for last {} batches = {}'.format(storing_frequency, np.mean(np.array(epoch_losses[-storing_frequency:]))))
          #print_time()
          #torch.save(basic_classifier.state_dict(), checkpoints_path+"/final_model_epoch_{}_{}.checkpoint".format(epoch, i+1))
  
    # Store after whole epoch
    print ('Epoch [{}/{}] finished with loss = {:.4f}'.format(epoch, num_epochs, np.mean(np.array(epoch_losses))))
    #torch.save(basic_classifier.state_dict(), checkpoints_path+"/final_model_epoch_{}.checkpoint".format(epoch))

In [ ]:
nn_pred = basic_classifier(torch.tensor(x_test.astype('float32')).to(device))

In [ ]:
nn_pred = nn_pred.flatten().detach().cpu().numpy().round()

In [ ]:
np.sum(nn_pred==y_test)/len(y_test)

In [ ]:
print(classification_report(y_test, nn_pred, digits=3))
print(confusion_matrix(y_test, nn_pred))